![University of Tehran](./img/UT.png)
#   <font color='red'><center>AI CA 1<center></font> 
## <center>Dr. Fadaei<center>
### <center>Daniyal Maroufi<center>
### <center>810098039<center>

# Aim

This assignment aims to solve a problem using Informed and Uninformed search algorithms such as BTS, IDS, and A* algorithm. Finally, we will compare the time complexity of these algorithms.

# Problem Definition

## Intial State

The initial state is the state that our agent will start from. In this case, Gandalf starts from his state at a given position in the input.

## Action

The agent can do an action to reach another state. In this case, Gandolf can have six actions: going up, right, down, and left and picking a friend, and placing them in another position.

## Transition Model

UP action: If it is permitted, the agent will move from state (x,y) to state (x-1,y)

DOWN action: If it is permitted, the agent will move from state (x,y) to state (x+1,y)

RIGHT action: If it is permitted, the agent will move from state (x,y) to state (x,y+1)

LEFT action: If it is permitted, the agent will move from state (x,y) to state (a,y-1)

PICK action: the agent will pick the friend

PLACE action: the agent will place the friend

## Goal State

The goal state is that the agent has successfully done its tasks. In this case, Gandalf must place all the friends in their goal positions and reach the final point (Gandor).

# Algorithms

## BFS

The Breadth-First Search (BFS) is an algorithm for traversing or searching tree or graph data structures. It explores all the nodes at the present depth before moving to the next depth level. BFS is complete, and it will give us an optimal solution with time complexity of $O(b^d)$.

## IDS

The IDS or Iterative Deepening Search is an algorithm based on DFS which controls the depth before expanding another node. IDS leads to an optimal solution. Unlike DFS, its time complexity is $O(bm)$.

## A*

A* algorithm searches for the shortest path between the initial and the final state. It is used in various applications, such as maps.
In maps, the A* algorithm calculates the shortest distance between the source (initial state) and the destination (final state).
$g(n)$ is the cost of reaching the current state, and $h(n)$ is the heuristic function that estimates the cost of getting to the final state from the current node $n$.
A* search is complete, and its time complexity is $O(b^d)$.

# Classes

In [10]:
import queue
import time
import copy

## Node

In [230]:
class Node(object):
    def __init__(self,position):
        self.state=tuple(position)
        self.parent=None
        self.path=''
        self.picked_RF=None
        self.placed_RFs=set()

    def __eq__(self, other):
        if self.state==other.state and self.path==other.path and self.picked_RF==other.picked_RF and self.placed_RFs==other.placed_RFs:
            return True
        return False

    def __hash__(self):
        return hash((self.state,str(self.placed_RFs),self.picked_RF))
    
    def getDist(self):
        return len(self.path)
    
    def setPlacedRFs(self,_placed_RFs):
        self.placed_RFs=_placed_RFs


In [231]:
def createChild(parent=Node((-1,-1))):
    child=Node(parent.state)
    child.parent=parent
    child.path=copy.deepcopy(parent.path)
    child.picked_RF=parent.picked_RF
    child.placed_RFs=copy.deepcopy(parent.placed_RFs)
    return child

## Problem

In [232]:
class Problem():
    def __init__(self, test_name):
        with open('./tests/'+test_name+'.txt') as f:
            self.n,self.m = tuple(map(int,f.readline().split()))
            self.gandalf_initial_pos = tuple(map(int,f.readline().split()))
            self.gandor = tuple(map(int,f.readline().split()))
            self.k,self.l = tuple(map(int,f.readline().split()))
            self.orks=[]
            self.RFs_initial_pos=[]
            self.RFs_goal_pos=[]
            for _ in range(self.k):
                x,y,c = tuple(map(int,f.readline().split()))
                print(x,y,c)
                self.orks.append((x,y,c))
            for _ in range(self.l):
                x,y = tuple(map(int,f.readline().split()))
                self.RFs_initial_pos.append((x,y))
            for _ in range(self.l):
                x,y = tuple(map(int,f.readline().split()))
                self.RFs_goal_pos.append((x,y))
            self.allRFs=dict(zip(self.RFs_initial_pos,self.RFs_goal_pos))
    
    def getInitialNode(self):
        node=Node(self.gandalf_initial_pos)
        return node
    
    def goalTest(self, node=Node((-1,-1))):
        print('gTest:',len(node.placed_RFs),self.l)
        if len(node.placed_RFs)==self.l and node.state==self.gandor:
            return True
        return False

    def isInRegionOfOrk(self,ork,node=Node((-1,-1))):
        if abs(ork[0]-node.state[0])+abs(ork[1]-node.state[1])<ork[2]:
            return True
        return False

    def countPresence(self,ork,node=Node((-1,-1))):
        presence=0
        while node is not None and self.isInRegionOfOrk(ork,node):
            presence+=1
            node=node.parent
        return presence

    def canGo(self,node=Node((-1,-1))):
        if node.state[0]<0 or node.state[0]>=self.n or node.state[1]<0 or node.state[1]>=self.m:
            return False
        for ork in self.orks:
            # count the number of RF's presence in the ork's region
            if self.countPresence(ork,node)>=ork[2]:
                return False
        return True
    
    def actions(self):
        return {'pick':'pick','place':'place','L':(0,-1),'R':(0,1),'U':(-1,0),'D':(1,0)}



In [233]:
test0=Problem('test_04')
test0.allRFs

3 1 2
2 6 1
6 4 2


{(0, 7): (5, 0), (7, 1): (5, 6)}

In [234]:
def bfs(problem=Problem('test_00')):
    frontier=queue.Queue()
    frontierSet=set() # saves the hash of the states in the frontier
    explored=set() # saves the hash of the states that are explored
    seen_states, unique_seen_states = 0, 0
    root=problem.getInitialNode()
    frontier.put(root)
    frontierSet.add(hash(root))
    if problem.goalTest(root):
        return 1,1,root.path
    while not frontier.empty():
        print('frontier:',[a for a in frontierSet])
        node=frontier.get()
        print(hash(node))
        frontierSet.remove(hash(node))
        explored.add(hash(node))
        print(node.path)
        for action,transition in problem.actions().items():
            if action == 'pick':
                print('path',node.path)
                print('pick?','picked:',node.picked_RF,'state:',node.state,'initiallist:',problem.RFs_initial_pos)
                if node.picked_RF is None and node.state in problem.RFs_initial_pos:
                    #  and problem.allRFs[node.state] not in node.placed_RFs
                    child_with_RF=createChild(node)
                    child_with_RF.picked_RF=problem.allRFs[node.state]
                    # child_with_RF=Node(node.state)
                    # child_with_RF.parent=node
                    # child_with_RF.path=copy.deepcopy(node.path)
                    # child_with_RF.picked_RF=problem.allRFs[node.state]
                    # child_with_RF.placed_RFs=node.placed_RFs
                    print('picked!!')
                    seen_states+=1
                    frontier.put(child_with_RF)
                    frontierSet.add(hash(child_with_RF))
            elif action == 'place':
                if node.state == node.picked_RF:
                    print('placed!!')
                    child=createChild(node)
                    child.placed_RFs.add(node.state)
                    # child.picked_RF=None
                    # node.placed_RFs.add(node.state)
                    # node.picked_RF=None
                    seen_states+=1
                    frontier.put(child)
                    frontierSet.add(hash(child))
                    print('placed!:)')
            else:
                # child=Node((node.state[0]+transition[0],node.state[1]+transition[1]))
                child=createChild(node)
                child.state=(node.state[0]+transition[0],node.state[1]+transition[1])
                if problem.canGo(child):
                    print('couldGo:',child.state)
                    # child.parent=node
                    # child.path=copy.deepcopy(node.path)+action
                    child.path+=action
                    # child.picked_RF=node.picked_RF
                    # child.setPlacedRFs(node.placed_RFs)
                    seen_states+=1
                    if hash(child) not in explored and hash(child) not in frontierSet:
                        unique_seen_states+=1
                        if problem.goalTest(child):
                            return seen_states, unique_seen_states, child.path
                        frontier.put(child)
                        frontierSet.add(hash(child))
    return 0,0,''





In [235]:
bfs(test0)

gTest: 0 2
frontier: [3024374161306415464]
3024374161306415464

path 
pick? picked: None state: (0, 0) initiallist: [(0, 7), (7, 1)]
couldGo: (0, 1)
gTest: 0 2
couldGo: (1, 0)
gTest: 0 2
frontier: [-8824530195423681563, 8130865612512295748]
8130865612512295748
R
path R
pick? picked: None state: (0, 1) initiallist: [(0, 7), (7, 1)]
couldGo: (0, 0)
couldGo: (0, 2)
gTest: 0 2
couldGo: (1, 1)
gTest: 0 2
frontier: [6594756863676917358, -8824530195423681563, 9031644221338856203]
-8824530195423681563
D
path D
pick? picked: None state: (1, 0) initiallist: [(0, 7), (7, 1)]
couldGo: (1, 1)
couldGo: (0, 0)
couldGo: (2, 0)
gTest: 0 2
frontier: [6594756863676917358, -7060944752781310039, 9031644221338856203]
6594756863676917358
RR
path RR
pick? picked: None state: (0, 2) initiallist: [(0, 7), (7, 1)]
couldGo: (0, 1)
couldGo: (0, 3)
gTest: 0 2
couldGo: (1, 2)
gTest: 0 2
frontier: [-7060944752781310039, -1874292850895683322, -4752394880993000626, 9031644221338856203]
9031644221338856203
RD
path RD
pi

KeyboardInterrupt: 

In [39]:
test0.RFs_goal_pos


[(5, 0), (5, 6)]

In [30]:
dd.path='R'

In [31]:
dd.path+='L'

In [32]:
dd.path

'RL'

In [33]:
dd.getDist()

2

In [223]:
aa=Node((0,0))
a=Node((1,0))
a.parent=aa
b=Node((2,0))
b.parent=a
c=Node((3,0))
c.parent=b
d=Node((4,0))
d.parent=c

# print('aa:',test0.countPresence(test0.orks[0],d))

.isInRegionOfOrk(test0.orks[0],d)
print('bb:',test0.canGo(d))



# a.placed_RFs.add((2,2))
# b.placed_RFs=a.placed_RFs
# b.placed_RFs.add((2,3))
# b.placed_RFs

bb: True
